In [2]:
pip install pandas nltk scikit-learn openpyxl

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [13]:
# تحميل الموارد المطلوبة
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# إعداد الستيمينغ والكلمات الشائعة
stemmer = nltk.stem.PorterStemmer()
stop_words = set(stopwords.words('english'))  # يمكنك تغيير اللغة حسب الحاجة

def clean_text(text):
    # إزالة الأحرف غير المرغوب فيها
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # تحويل النص إلى حروف صغيرة
    text = text.lower()
    
    # تقسيم النص إلى كلمات
    tokens = word_tokenize(text)
    
    # إزالة الكلمات الشائعة
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # تطبيق الستيمينغ
    stemmed_words = [stemmer.stem(word) for word in filtered_tokens]
    
    return ' '.join(stemmed_words)

# قراءة البيانات من ملف Excel
file_path = r'C:\Users\hp\Desktop\‏‏‏‏BBC news.xlsx'  # تأكد من تعديل المسار حسب الحاجة
data = pd.read_excel(file_path)

# افترض أن العمود الذي يحتوي على النص هو "text"
data['final_result'] = data['text'].apply(clean_text)

# عرض النتائج
print(data[['text', 'final_result']])

# استخراج الميزات باستخدام Bag of Words
bow_vectorizer = CountVectorizer(max_features=1000)  # يمكنك تعديل العدد حسب الحاجة
bow_matrix = bow_vectorizer.fit_transform(data['final_result'])
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=bow_vectorizer.get_feature_names_out())

# استخراج الميزات باستخدام TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # يمكنك تعديل العدد حسب الحاجة
tfidf_matrix = tfidf_vectorizer.fit_transform(data['final_result'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# دمج الميزات في DataFrame واحد
combined_df = pd.concat([data[['final_result']], bow_df, tfidf_df], axis=1)

# حفظ الميزات في ملف Excel واحد
output_file = r'C:\Users\hp\Desktop\final.xlsx'  # تأكد من تعديل المسار حسب الحاجة
combined_df.to_excel(output_file, index=False)
print("تم حفظ ميزات Bag of Words وTF-IDF في الملف:", output_file)

# إعداد المتغيرات للنموذج
X = data['final_result']  # النصوص
y = data['category']  # التسميات (تأكد من وجود عمود التسميات)

# تقسيم البيانات إلى مجموعة تدريب ومجموعة اختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تحويل النصوص إلى ميزات باستخدام TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# تصنيف باستخدام SVM
svm_model = SVC(kernel='linear')  # يمكنك تعديل النواة حسب الحاجة
svm_model.fit(X_train_tfidf, y_train)
svm_predictions = svm_model.predict(X_test_tfidf)

# تصنيف باستخدام Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)
nb_predictions = nb_model.predict(X_test_tfidf)

# تصنيف باستخدام Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
rf_predictions = rf_model.predict(X_test_tfidf)

# تقييم النماذج
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_predictions))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))

# طباعة تقرير التصنيف
print("\nSVM Classification Report:\n", classification_report(y_test, svm_predictions))
print("\nNaive Bayes Classification Report:\n", classification_report(y_test, nb_predictions))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, rf_predictions))

# عرض أحجام المجموعات
print("حجم مجموعة التدريب:", X_train.shape[0])
print("حجم مجموعة الاختبار:", X_test.shape[0])

                                                   text  \
0     mutant book wins guardian prize a book about t...   
1     running around the olympics it was back to off...   
2     parker misses england clash tom shanklin will ...   
3     munster cup tie switched to spain munster s he...   
4     cyber crime booms in 2004 the last 12 months h...   
...                                                 ...   
2220  smart search lets art fans browse if you don t...   
2221  clijsters hope on aussie open kim clijsters ha...   
2222  mps quiz aides over royal income senior offici...   
2223  podcasters  look to net money nasa is doing it...   
2224  bargain calls widen softbank loss japanese com...   

                                           final_result  
0     mutant book win guardian prize book evolut mut...  
1     run around olymp back offici duti last week ro...  
2     parker miss england clash tom shanklin start c...  
3     munster cup tie switch spain munster heineken ...  
4